In [100]:
class Node:
    def __init__(self, data, depth=0, lchild=None, rchild=None):
        self.data = data
        self.depth = depth
        self.lchild = lchild
        self.rchild = rchild


class KdTree:
    def __init__(self):
        self.KdTree = None
        self.n = 0
        self.nearest = None

    def create(self, dataSet, depth=0):
        if len(dataSet) > 0:
            m, n = np.shape(dataSet)
            self.n = n - 1
            axis = depth % self.n
            mid = int(m / 2)
            dataSetcopy = sorted(dataSet, key=lambda x: x[axis])
            node = Node(dataSetcopy[mid], depth)
            if depth == 0:
                self.KdTree = node
            node.lchild = self.create(dataSetcopy[:mid], depth+1)
            node.rchild = self.create(dataSetcopy[mid+1:], depth+1)
            return node
        return None

    def preOrder(self, node):
        if node is not None:
            print(node.depth, node.data)
            self.preOrder(node.lchild)
            self.preOrder(node.rchild)

    def search(self, x, count=1):
        nearest = []
        for i in range(count):
            nearest.append([-1, None])
        self.nearest = np.array(nearest)

        def recurve(node):
            if node is not None:
                axis = node.depth % self.n
                daxis = x[axis] - node.data[axis]
                if daxis < 0:
                    recurve(node.lchild)
                else:
                    recurve(node.rchild)

                dist = sqrt(sum((p1 - p2) ** 2 for p1, p2 in zip(x, node.data)))
                for i, d in enumerate(self.nearest):
                    if d[0] < 0 or dist < d[0]:
                        self.nearest = np.insert(self.nearest, i, [dist, node], axis=0)
                        self.nearest = self.nearest[:-1]
                        break

                n = list(self.nearest[:, 0]).count(-1)
                if self.nearest[-n-1, 0] > abs(daxis):
                    if daxis < 0:
                        recurve(node.rchild)
                    else:
                        recurve(node.lchild)

        recurve(self.KdTree)

        knn = self.nearest[:, 1]
        belong = []
        for i in knn:
            belong.append(i.data[-1])
        b = max(set(belong), key=belong.count)

        return self.nearest, b

In [101]:
import numpy as np
from math import sqrt
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.utils import shuffle
min_max_scaler=preprocessing.MinMaxScaler()
from sklearn.datasets import load_iris
iris = load_iris()
iris_sample = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_sample = pd.DataFrame(min_max_scaler.fit_transform(np.array(iris_sample)))
iris_sample['label'] = iris.target
iris_sample= iris_sample.drop_duplicates()

In [102]:
def run(data):  
    #print('epoch..........:',term)
    
    Dn,Dn_filted = [],[]
    for x in data:
        #plt.scatter(x[0], x[1], c='red', marker='x')  # 测试点
        near, belong = kdt.search(x[:-1], k+1)  # 设置临近点的个数
        density = 0
        t,hon,hen,DRL = 0,0,0,0
        for n in near:
            if x[-1] == n[1].data[-1] and (x != n[1].data).any():
                t += 1
                hon += 1

            elif  x[-1] != n[1].data[-1]:
                hen += 1

            density += n[0]  #密度   

        DRL = (hen - hon) / k  #标签异同差比    
        Ps = t / k
        Pd = 1 - Ps
        if (Ps==1)|(Ps==0):
            H = 0
        else:
            H = -Ps* np.log2(Ps)-Pd*np.log2(Pd)

       # print(densit)
        if (density <= a): #高密度区域
           # print('高密度区域')
      
            if DRL>0:
                Dn.append(list(x))
              
                #print('高密度区域噪声')
        else:
            if (H < H_): #低密度单一标签区域
            
                if DRL>0: #再次判断
                    Dn.append(list(x))
            else: #低密度混合标签区域
              #  print('混合区')
                data_hon,data_hen=[],[]
                for i in data:
                    if i[-1]==x[-1]:
                        data_hon.append(i)
                    else:
                        data_hen.append(i)
                data_positive=np.array(data_hon)
                data_negetive=np.array(data_hen)
                kdt_hon=KdTree()
                kdt_hen=KdTree()
                kdt_hon.create(data_positive)
                kdt_hen.create(data_negetive)
                near_hon,belong_hon=kdt_hon.search(x[:-1],k+1)
                near_hen,belong_hen=kdt_hen.search(x[:-1],k)
                dist_hon,dist_hen = 0,0
               # print('同类近邻')
                for n in near_hon:
                    dist_hon += n[0]
                    #print(n[1].data,n[0])
                #print('异类近邻')
                for n in near_hen:
                    dist_hen += n[0]
                   # print(n[1].data,n[0])
                    
                if (dist_hon ) / (dist_hen ) > 1.0:
                    Dn.append(list(x))
                   # print('低密度混合标签区域噪声')  
                   # print(x)
    noise_neighbor=[]                   
    for e in Dn:  # 计算噪声分数
        e=np.array(e)
        near3,belong3=kdt.search(e[:-1],k+1) #k近邻
        for n3 in near3:
            noise_neighbor.append(list(n3[1].data))
    for e in Dn:
        e=np.array(e)
        near4,belong4=kdt.search(e[:-1],k+1)
        t_e=noise_neighbor.count(list(e))-1 #处于其它噪声近邻的次数
        confidence_e= 1 / np.sqrt(1+t_e*t_e)#e的置信度
        for i in range(1,k+1):
            t_ei=noise_neighbor.count(list(near4[i][1].data))
            confidence_ei=1 / np.sqrt(1+t_ei*t_ei) 
            if near4[i][1].data[-1]==e[-1]:
                different_class=-1
            else:
                different_class=1
            if list(near4[i][1].data) in Dn: 
                isnoise=1
            else:
                isnoise=-1
            near5,belong5=kdt.search(near4[i][1].data[:-1],k+1)
            neighborhood_e, dis_neighbor,dis_noise=0,0,0
            for j in range(1,k+1):
                dis_neighbor += near5[j][0]
                if list(near5[j][1].data) in Dn:
                    dis_noise +=near5[j][0]
            clean_ei= (dis_neighbor + isnoise*(dis_noise-dis_neighbor)) / (2*dis_neighbor) #纯净度
            neighborhood_e += (clean_ei*confidence_ei*different_class)/k 
        NS_e=confidence_e * neighborhood_e #噪声得分
        if NS_e > 0:
            Dn_filted.append(list(e))
                   
    #print('标签噪声：',Dn)
    noise_list=noise.values.tolist()
    TP = 0
    for c in Dn_filted:
        if c in noise_list:
            TP += 1
    FP = len(Dn_filted) - TP
    FN = len(noise_list) - TP
    precision= TP / (TP + FP)
    recall= TP / (TP + FN)
    fh= 1.25* (precision * recall) / (0.25*precision + recall) #F0.5值
    fh= 2*precision*recall / (precision+recall)
    re = len(Dn_filted) / len(data)
    false_re= FP / len(data)#错误移除比率
   # print('准确率:',precision)
  #  print('召回率:',recall)
   # print('F0.5值:',fh)
   # print('移除率:',re)
   # print('错误移除率:',false_re)
    return fh,re,false_re

In [103]:
def run2(data):  
    #print('epoch..........:',term)
    Dn = []
    for x in data:
        #plt.scatter(x[0], x[1], c='red', marker='x')  # 测试点
        near, belong = kdt.search(x[:-1], k+1)  # 设置临近点的个数
        #print("test:")
       # print(x, "predict:", belong)
       # print("nearest:")
        density = 0
        t,hon,hen,dist_hon,dist_hen,DoD,DRL= 0,0,0,0,0,0,0
        for n in near:
            if x[-1] == n[1].data[-1] and (x != n[1].data).any():
                t += 1
                hon += 1
                dist_hon += n[0]

            elif  x[-1] != n[1].data[-1]:
                hen += 1
                dist_hen += n[0]

            density += n[0]  #密度   

        DoD = abs(dist_hen - dist_hon) #相异性差值(要加绝对值)
        DRL = (hen - hon) / k  #标签异同差比    
        Ps = t / k
        Pd = 1 - Ps
        if (Ps==1)|(Ps==0):
            H = 0
        else:
            H = -Ps* np.log2(Ps)-Pd*np.log2(Pd)
      #  print('densit:',density)
       
        if (density <= a): #高密度区域
            print('高密度区域')
            if DRL > 0:
                Dn.append(list(x))
              
                #print('高密度区噪声')
        else:
            if (H < H_): #低密度单一标签区域
                if DRL > 0:
                    Dn.append(list(x))
                 
                #    print('低密度单一标签噪声')
            else: #低密度混合标签区  
        
             #   print('低密度混合区')
                DoD_ =  density / k
               # print('DoD阈值:',DoD_)
                if DoD >= DoD_:
                    hon_sa,hen_sa,DRL_sa=0,0,0
                    near2=[]
                    for n in near:
                        if n[0] <= a: #该点到待测点距离小于a
                            near2.append(n)
                    for n2 in near2:
                        if (x[-1]==n2[1].data[-1])and (x != n2[1].data).any():
                            hon_sa +=1
                        elif x[-1] !=n2[1].data[-1]:
                            hen_sa +=1
                    DRL_sa = hen_sa - hon_sa
                    if DRL_sa > 0:        
                        Dn.append(list(x))
                      #  print('低密度混合标签区噪声')
                       # print(x)

                    elif DRL_sa ==0 and x[-1] != near[len(near2)][1].data[-1]:
                        Dn.append(list(x))    
                       # print('低密度混合标签区噪声')
                       # print(x)               
    noise_list=noise.values.tolist()
    TP = 0
    for c in Dn:
        if c in noise_list:
            TP += 1
    FP = len(Dn) - TP
    FN = len(noise_list) - TP
    precision= TP / (TP + FP)
    recall= TP / (TP + FN)
    fh= 1.25* (precision * recall) / (0.25*precision + recall) #F0.5值
    #fh= 2*precision*recall / (precision+recall)
    re = len(Dn) / len(data) #移除率
    false_re= FP / len(data)
    #print('准确率:',precision)
    #print('召回率:',recall)
    #pri#t('F0.5值:',fh)
    #print('移除率:',re)
   # print('错误移除率:',false_re)
    return fh,re,false_re

In [104]:
%%time
df=iris_sample
data2=np.array(df.drop('label',axis=1))
dist2=[]
for m in range(0,len(data2)-1):
    for n in range(m+1,len(data2)):
        dist2.append(sqrt(sum((data2[m] - data2[n]) ** 2 )))
dist2.sort(reverse=True)
dist2=np.array(dist2)
location= (len(dist2) - 1) * 0.1
zhenshu=int(location) #整数部分
xiaoshu=(round(location,1)-zhenshu) #小数部分
a = (1-xiaoshu)*dist2[zhenshu] + xiaoshu*dist2[zhenshu+1]#密度阈值(%k分位数)
print('a值：',a)
#a = np.percentile(dist2,10) #密度阈值(%10分位数)
k=7
x1 = np.array([[1,1],[1,-1]])
x2 = np.array([1,np.floor(k / 3)/ k])
(Ps_,Pd_)=np.linalg.solve(x1,x2)
H_ = -Ps_* np.log2(Ps_)-Pd_*np.log2(Pd_) #标签混合程度阈值

F_list,Re_list,F_list2,Re_list2,Fre_list,Fre_list2=[],[],[],[],[],[]
#for rate in[0.05,0.1,0.2,0.3,0.4]:
for rate in [0.4]:
    Fh,Re,Fh2,Re2,false_re,false_re2=[],[],[],[],[],[]
    for term in range(1): #多次试验取均值

        noise_set=pd.DataFrame(columns=df.columns)
        for r in range(0,(df.label.max()+1)):    
            noise_set=pd.concat([noise_set,df[df.label==r].sample(frac=rate, replace=False,random_state=42)]) #选取噪声比例  
        #bb=df[df.label==1].sample(frac=rate, replace=False) #选取噪声比例
        #cc=df[df.label==2].sample(frac=rate, replace=False) #选取噪声比例

        data_cut=df[~df.index.isin(noise_set.index)]
        no=[]
        for j in range(0,len(noise_set)):
            no.append(int(np.random.rand(1)[0]*(df.label.max()+1))%10)
        noise_set['label2']=no
        noise_set.reset_index(drop=True,inplace=True)
        for l in range(0,len(noise_set)):  #随机替换标签
            label_set=list(set(df.label))
            if (noise_set.loc[l,'label']==noise_set.loc[l,'label2']):
                label_set.remove(noise_set.loc[l,'label'])
                noise_set.loc[l,'label2']=np.random.choice(label_set) #从剩余列表中随机选择
        noise=noise_set.drop(['label'],axis=1)
        noise.rename(columns={'label2':'label'}, inplace = True)
        data= pd.concat([data_cut,noise],axis=0,ignore_index=True) #含噪声数据集
        data=np.array(data)
        kdt = KdTree()
        kdt.create(data)
        
        result=run(data)
        result2=run2(data)
        Fh.append(result[0])
        Re.append(result[1])
        Fh2.append(result2[0])
        Re2.append(result2[1])
        false_re.append(result[2])
        false_re2.append(result2[2])
    F_list.append(np.mean(Fh))
    Re_list.append(np.mean(Re))
    Fre_list.append(np.mean(false_re))
    F_list2.append(np.mean(Fh2))
    Re_list2.append(np.mean(Re2))
    Fre_list2.append(np.mean(false_re2))

a值： 1.1758578432296467
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
高密度区域
Wall time: 2.92 s


In [105]:
#run(np.array(noise))  1.1758578432296467

In [106]:
#run2(np.array(noise))

In [107]:
import mpl_toolkits.axisartist.axislines as axislines
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = [u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib

Using matplotlib backend: Qt5Agg


In [108]:
noise_rate=[5,10,20,30,40]
fig = plt.figure(figsize=(6,14))
ax1 = axislines.Subplot(fig, 2,1,1)
fig.add_subplot(ax1)
ax1.set_xticks(noise_rate)
ax1.axis([0,40,0,0.4])
plt.plot(noise_rate,Fre_list,'ro-',label='IM_DDF')
plt.plot(noise_rate,Fre_list2,'b*-.',label='DDF')
plt.xlabel('噪声比例/%')
#plt.ylabel('F0.5')
plt.ylabel('false_re')
#plt.title('不同噪声下F0.5值',fontsize=12)
#plt.title('(image)不同噪声下错误移除比例',fontsize=12)
plt.legend()
plt.show()
ax2 = axislines.Subplot(fig, 2,1,2)
fig.add_subplot(ax2)
ax2.set_xticks([5,10,20,30,40])
ax2.set_yticks([0,0.2,0.4,0.6,0.8,1])
ax2.axis([0,40,0,1])
plt.plot(noise_rate,Re_list,'ro-',label='IM_DDF')
plt.plot(noise_rate,Re_list2,'b*-.',label='DDF')
plt.xlabel('噪声比例/%')
#plt.ylabel('Re')
#plt.title('不同噪声下的过滤值Re',fontsize=12)
plt.legend()
plt.tight_layout(6) #设置子图间隔
plt.suptitle('(Iris)')
plt.show()

ValueError: x and y must have same first dimension, but have shapes (5,) and (1,)

In [ ]:
Re_list,Re_list2

In [ ]:
F_list,F_list2